# Linear Algebra: Manipulating images with matrix operations
First, we load an image using the Python Image Processing Library. Note that even though we'll reimplement most of the library's functionality (crop, resize, scale, merge methods), we're not reimplementing image reading, as that would mean interpreting bytes based on the image spec (JPEG/PNG/whatever), and that isn't our focus.

In [5]:
import math
import numpy as np
from PIL import Image

image = Image.open('fixtures/koala.jpg')

# This works because the Image class implements .__array_interface__()
matrix = np.array(image)

# It returns a 3D array (Would have been nicer if it returned a 2D array of 3-tuples)
print(matrix.shape)
print(matrix[0][0])


(768, 1024, 3)
[101  90  58]


## Understanding the image
The image is 1024 by 768, in RGB mode. Its matrix has the shape (768, 1024, 3). So this is a 2-dimensional matrix (768 * 1024) with 3 items each, representing the RGB value of each pixel. `matrix[i][j]` will be `[red_value  green_value  blue_value]`.

## Challenge 1: Size manipulation - Crop
Let's say we want to crop the image to a central portion half the original area, and maintaining the shape (ie the aspect ratio).

Area of image = 1024 * 768 = 786 432 pixels².
Desired area = 393 216 pixels²
To find the length and breadth to give us this area while maintaining the aspect ratio: 4p * 3p = 12p² = 393216 => p = 181.02.

This gives us dimensions 4(181.02) by 3(181.02) = 724 by 543 (round to 544 to be an even number).

If we call the 768 dimension x and the 1024 dimension y, this means we will keep the inner 544 in x (remove 112 at the start and end) and the inner 724 in y (remove 150 at the start and end).

In [33]:
cropped = matrix[112:-112]
cropped = cropped[:, 150:-150]

Image.fromarray(cropped).show()

Alternatively, if we want to instead crop to a specific aspect ratio, say 1:1 (as for a profile picture). The resulting dimensions will then be 768 by 768, since we can only crop to the smaller dimension. If we still want to crop outwards from the centre, this means we'll take the full 768 in x and keep 768 from y (remove 128 from each end).

In [6]:
cropped = matrix[:, 128:-128]
Image.fromarray(cropped).show()

Next challenge: cropping to a circular shape? To do this, we'll find all pixels outside the circle and zero them. First, the radius of the circle is 768/2 = 384. We can go over each cell and zero it if its distance from the centre is greater than 384.
The centre is: [383.5,383.5]

In [19]:
radius = (cropped.shape[0] + 1)/2 if cropped.shape[0] % 2 == 0 else cropped.shape[0]/2
centre = (radius, radius)

circular_cropped = np.array(cropped)
for i in range(cropped.shape[0]):
    for j in range(cropped.shape[1]):
        distance = np.linalg.norm(np.array([i, j]) - centre)
        if distance > radius:
            circular_cropped[i,j] = [0,0,0]

Image.fromarray(circular_cropped).show()


## Challenge 2: Size manipulation - Resize
How do we reduce the size of this image (without changing its other dimensions)?
- We need to keep its original aspect ratio, which is 4:3 (1024:768)
- To reduce this, we must get rid of some pixels. We can't simply take the first/last 600. We must reduce proportionally.

Supposing we want to reduce it to 600x400. This is a reduction of 1.28. So we must replace every 1.28 pixels by 1 pixel, or every 64 pixels must be replaced by 50.

In [49]:
scaling_factor = 600/1024
# scaling_factor = 1920/1024 # Weakness: empty pixels (solve: fill in neighbouring pixels)
scaling_matrix = np.array([[scaling_factor, 0], [0, scaling_factor]])
new_shape = np.matmul(scaling_matrix, matrix.shape[0:2])
# todo figure out how to cast with numpy
new_shape = (int(new_shape[0]), int(new_shape[1]), matrix.shape[2])
scaled = np.empty(new_shape, dtype=matrix.dtype)

for i1 in range(matrix.shape[0]):
    for j1 in range(matrix.shape[1]):
        i2, j2 = np.matmul(scaling_matrix, [i1, j1])
        scaled[int(i2), int(j2)] = matrix[i1, j1]

Image.fromarray(scaled).show()


## Challenge 3: Geometrical manipulation - Rotate


In [27]:
theta = np.radians(3 * np.pi / 2)
# rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
rotation_matrix = np.array([[0, 1], [-1, 0]])

new_shape = np.matmul(rotation_matrix, matrix.shape[0:2])
rotated = np.empty((*np.abs(new_shape), matrix.shape[2]), dtype=matrix.dtype)

for i1 in range(matrix.shape[0]):
    for j1 in range(matrix.shape[1]):
        i2, j2 = np.matmul(rotation_matrix, [i1, j1])
        rotated[i2][j2] = matrix[i1, j1]

Image.fromarray(rotated).show()



## Challenge 3: Geometrical manipulation - Reflect
To reflect geometrically around the y-axis, the mapping is f: x -> -x. Here, x is the coordinate (eg 1024/768). So we only need to invert the indexes.

 

In [6]:
reflected_in_y = matrix[:, ::-1]
reflected_in_x = matrix[::-1]
reflected_in_xy = matrix[::-1, ::-1]
Image.fromarray(reflected_in_xy).show()


## Challenge 4: Colour manipulation - Filter

In [40]:
print(matrix[0, 0])
filtered = matrix
# We could also just do filtered[:,:,2] *= 0.5, but this fails because of the casting (float to int)
filtered[:, :, 2] = np.multiply(0.5, filtered[:, :, 2], casting='unsafe')
print(filtered[0, 0])

Image.fromarray(filtered).show()


[101  90  29]
[101  90  14]
